## AutoGen Swarm - Travel Scenario
Swarm implements a team in which agents can hand off task to other agents based on their capabilities. It is a multi-agent design pattern first introduced by OpenAI in an experimental project. The key idea is to let agent delegate tasks to other agents using a special tool call, while all agents share the same message context. This enables agents to make local decisions about task planning, rather than relying on a central orchestrator such as in SelectorGroupChat.

The speaker agent is selected based on the most recent HandoffMessage message in the context. This naturally requires each agent in the team to be able to generate HandoffMessage to signal which other agents that it hands off to.

In general, this is how it works:

1. Each agent has the ability to generate HandoffMessage to signal which other agents it can hand off to. For AssistantAgent, this means setting the handoffs argument.

2. When the team starts on a task, the first speaker agents operate on the task and make locallized decision about whether to hand off and to whom.

3. When an agent generates a HandoffMessage, the receiving agent takes over the task with the same message context.

4. The process continues until a termination condition is met.

## Load Azure Configurations

In [2]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client

In [3]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)


## Creating the Agents

In [4]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

# create the planning agent
planning_agent = AssistantAgent(
    "planning_agent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    handoffs=["user_proxy", "places_agent", "weather_agent", "hotel_agent", "restaurant_agent", "writer_agent"],
    model_client=az_model_client,
    system_message=f"""
    You are a planning agent that coordinates the work of your team members.
    Coordinate handoff to specialized agents:
    - weather_agent to get the information from the web about the weather
    - places_agent to look for places to visit
    - hotel_agent to look for available hotels
    - restaurant_agent to look for available restaurants
    - writer_agent to write the final plan
    - final handoff to user proxy for final approval

     You will follow this sequence:
        Step 1: Weather Agent will provide information about the weather.
        Step 2: Places Agent will provide information about places to visit based on the weather.
        Step 3: Hotel Agent will provide information about available hotels.
        Step 4: Restaurant Agent will provide information about available restaurants.
        Step 5: Writer Agent will write the final plan.
        Step 6: User Proxy will provide feedback on process.

    Always send your plan first, then handoff to appropriate agent.
    Always handoff to a single agent at a time.
    """,
)


In [5]:
from datetime import datetime
from autogen_agentchat.teams import RoundRobinGroupChat

async def get_weather(city: str):
    return f"The weather in {city} is 4 degrees celsius and cloudy."


# Creating the weather agent
weather_agent = AssistantAgent(
    name="weather_agent",
    description="This agent will provide information about the weather based on the city it is given",
    handoffs=["places_agent"],
    model_client=az_model_client,
    tools=[get_weather],
    system_message="""
    You are the weather agent. You will provide information about the weather based on the city you are given.
    Always handoff back to places agent when complete.
    """,
)


In [10]:
# Creating the places agent
places_agent = AssistantAgent(
    name="places_agent",
    description="This agent will provide information about places to visit based on the weather",
    handoffs=["planning_agent"],
    model_client=az_model_client,
    tools=[get_weather],
    system_message="""
    You are the places agent. You will provide information about places to visit based on the weather.
    Always handoff back to planning agent when complete.
    """,
)

# Creating the hotel agent
hotel_agent = AssistantAgent(
    name="hotel_agent",
    description="This agent will provide information about available hotels",
    handoffs=["planning_agent"],
    model_client=az_model_client,
    tools=[get_weather],
    system_message="""
    You are the hotel agent. You will provide information about available hotels.
    Always handoff back to planning agent when complete.
    """,
)

# Creating the hotel agent
restaurant_agent = AssistantAgent(
    name="restaurant_agent",
    description="This agent will provide information about available restaurants",
    handoffs=["planning_agent"],
    model_client=az_model_client,
    tools=[get_weather],
    system_message="""
    You are the restaurant agent. You will provide information about available restaurants.
    Always handoff back to planning agent when complete.
    """,
)

# Define a tool to write the content to a file
async def write_to_file(content: str) -> str:
    # Write the content to a file
    with open("../Data/agagents-output/output.txt", "w") as file:
        file.write(content)

# Creating the writer agent
writer_agent = AssistantAgent(
    name="writer_agent",
    description="This agent will write the final plan combining all the information from the other agents",
    handoffs=["planning_agent"],
    model_client=az_model_client,
    tools=[write_to_file],
    system_message="""
    You are the writer agent. You will write the final plan combining all the information from the other agents.
    You will write the final output to a file.
    Always handoff back to planning agent when complete.
    """,
)

In [11]:
# Create the user proxy agent.
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

## Creating the Team

In [12]:
text_mention_termination = TextMentionTermination("APPROVE")
max_messages_termination = MaxMessageTermination(max_messages=1000)
termination = text_mention_termination | max_messages_termination

team = Swarm(
    participants=[planning_agent, user_proxy, weather_agent, places_agent, hotel_agent, restaurant_agent, writer_agent], termination_condition=termination
)

## Specify the Task and Run the Team

In [14]:
task = "I want to travel to New York for 3 days. Can you help me plan my trip?"
response = await Console(team.run_stream(task=task))

---------- user ----------
I want to travel to New York for 3 days. Can you help me plan my trip?
---------- planning_agent ----------
[FunctionCall(id='call_2zClNBvuCBnvHG7KD49Zob45', arguments='{}', name='transfer_to_restaurant_agent')]
---------- planning_agent ----------
[FunctionExecutionResult(content='Transferred to restaurant_agent, adopting the role of restaurant_agent immediately.', call_id='call_2zClNBvuCBnvHG7KD49Zob45')]
---------- planning_agent ----------
Transferred to restaurant_agent, adopting the role of restaurant_agent immediately.
---------- restaurant_agent ----------
[FunctionCall(id='call_KtD23wzB1AA3BHJvTkQf7hJw', arguments='{}', name='transfer_to_planning_agent')]
---------- restaurant_agent ----------
[FunctionExecutionResult(content='Transferred to planning_agent, adopting the role of planning_agent immediately.', call_id='call_KtD23wzB1AA3BHJvTkQf7hJw')]
---------- restaurant_agent ----------
Transferred to planning_agent, adopting the role of planning_age

## Printing whole response per agent source

In [35]:
# ANSI escape code for bold text
bold_start = "\033[1m"
bold_end = "\033[0m"

# ANSI escape code for red text
red_start = "\033[31m"
red_end = "\033[0m"

for messages in response.messages:
    source = messages.source
    print(f"{bold_start}{red_start}{source}{bold_end}{red_end}")
    print(messages.content)

user
I want to travel to New York for 3 days. Can you help me plan my trip?
planning_agent
[FunctionCall(id='call_UtzJTJoTDV8hxsixxtGVtEBc', arguments='{}', name='transfer_to_weather_agent')]
planning_agent
[FunctionExecutionResult(content='Transferred to weather_agent, adopting the role of weather_agent immediately.', call_id='call_UtzJTJoTDV8hxsixxtGVtEBc')]
planning_agent
Transferred to weather_agent, adopting the role of weather_agent immediately.
weather_agent
[FunctionCall(id='call_rRkvjzCT3LMKqJjHVPSn3p5c', arguments='{"city":"New York"}', name='get_weather')]
weather_agent
[FunctionExecutionResult(content='The weather in New York is 4 degrees celsius and cloudy.', call_id='call_rRkvjzCT3LMKqJjHVPSn3p5c')]
weather_agent
The weather in New York is 4 degrees celsius and cloudy.
weather_agent
[FunctionCall(id='call_LWpsAOMG0bwgUVgvvT8LYNM6', arguments='{}', name='transfer_to_places_agent')]
weather_agent
[FunctionExecutionResult(content='Transferred to places_agent, adopting the ro